# Data Analytics - Bus Data Understanding and Prep

## Audit

Author: Danning

Last Modified By: Adam

Module: COMP47630

DC:     2021-06-20

DLM:    2021-06-23

Desc:   This file contains an analysis of the historic bus data

Dict:   The Data Dictionary for the Data Set is available in Brightspace

## Table of Contents

00. Introduction

01. Exec Summary and Results

02. Modules

03. Constants

04. Ingestion

05. Cleansing


## 00. Introduction

### 00.01 Background
(here)

### 00.02 Problem Scope
(here)

### 00.03 Data
(here)

### 00.04 Approach
(here)

## 01. Exec Summary

(here)

---
# ------BEGIN---------- #

## 01. Static

### 01.01 Modules
Import all modules here

In [1]:
import sys
print(sys.executable)

/home/team10/comp47360/bin/python3


In [2]:
####--------------------------------------
#00.Import Modules
####--------------------------------------

######---------BEGIN
#      SUPPRESS DEPRECIATION WARNINGS: Applicable to datetime_is_numeric=True
######--------END

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

######---------BEGIN
#      ML
######--------END

#import nltk as nl
import sklearn as sk
import matplotlib as mp
#import xgboost as xg
#import pymc3 as pymc
#import sympy as sym



######---------BEGIN
#      SQL/API
######--------END


#import requests as rq
#import sqlalchemy as sqla
#import pyodbc
#import cx_oracle as cx


######---------BEGIN
#     GENERAL
######--------END

import pandas as pd
import datetime as dt
import numpy as np
import sys
import os
from dask import dataframe as dask_df
#import pyspark as spk
#import json
#import time
#import socket
#import traceback as tb
#import platform
#from psutil import virtual_memory
import pickle as pck


######---------BEGIN
#     VISUALISATIONS
######--------END


import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates


#For showing plots directly in the notebook run the command below
#%matplotlib inline


###HTML Output Hiding
#Install hide input extension
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable hide_input_all/main
#!jupyter nbextension enable hide_input/main
#!jupyter nbextension enable codefolding/main

#!jupyter nbextension disable hide_input_all/main
#!jupyter nbextension disable hide_input/main
#!jupyter nbextension disable codefolding/main

#Update with Filename: Run in Terminal, post completion, after hiding all cells for report
#!jupyter nbconvert --to=html bus_Data.ipynb


### 01.02 Constants
Import all Constants here

In [3]:
data_year='2018'
bus_leavetimes_filepath="./data/rt_leavetimes_DB_{}.txt".format(data_year)
bus_trips_filepath="./data/rt_trips_DB_{}.txt".format(data_year)

bus_leavetimes_sep=";"
bus_trips_sep=";"

bus_leavetimes_data_dictionary={
        'DATASOURCE':['Description','category']
        ,'DAYOFSERVICE':['Description','datetime']
        ,'TRIPID':['Description','category']
        ,'PROGRNUMBER':['Description','category']
        ,'STOPPOINTID':['Description','category']
        ,'PLANNEDTIME_ARR':['Description','int64']
        ,'PLANNEDTIME_DEP':['Description','int64']
        ,'ACTUALTIME_ARR':['Description','int64']
        ,'ACTUALTIME_DEP':['Description','int64']
        ,'VEHICLEID':['Description','category']
        ,'PASSENGERS':['Description','float64']
        ,'PASSENGERSIN':['Description','float64']
        ,'PASSENGERSOUT':['Description','float64']
        ,'DISTANCE': ['Description','float64']
        ,'SUPPRESSED':['Description','category']
        ,'JUSTIFICATIONID':['Description','category']
        ,'LASTUPDATE':['Description','datetime']
        ,'NOTE': ['Description','string']
                        }


lt_datetime_columns=[column_headers for column_headers, column_desc_array in bus_leavetimes_data_dictionary.items() if column_desc_array[1] == 'datetime']
lt_categorical_columns=[column_headers for column_headers, column_desc_array in bus_leavetimes_data_dictionary.items() if column_desc_array[1] == 'category']
lt_num_columns=[column_headers for column_headers, column_desc_array in bus_leavetimes_data_dictionary.items() if column_desc_array[1] in ('numeric','int64','float64')]



bus_leavetimes_metadata_dictionary={}

for column_headers, column_desc_array in bus_leavetimes_data_dictionary.items():
    bus_leavetimes_metadata_dictionary[column_headers]=column_desc_array[1]
    
    
bus_trips_data_dictionary={
        'DATASOURCE':['Description','category']
        ,'DAYOFSERVICE':['Description','datetime']
        ,'TRIPID':['Description','category']
        ,'LINEID':['Description','category']
        ,'ROUTEID':['Description','category']
    
        ,'DIRECTION':['Description','category']
        
        ,'PLANNEDTIME_ARR':['Description','int64']
        ,'PLANNEDTIME_DEP':['Description','int64']
        ,'ACTUALTIME_ARR':['Description','int64']
        ,'ACTUALTIME_DEP':['Description','int64']
    
        ,'BASIN':['Description','category']
        ,'TENDERLOT':['Description','float64']
    
        ,'SUPPRESSED':['Description','category']
        ,'JUSTIFICATIONID':['Description','category']
        ,'LASTUPDATE':['Description','datetime']
        ,'NOTE': ['Description','string']
                        }

bus_trips_metadata_dictionary={}

for column_headers, column_desc_array in bus_trips_data_dictionary.items():
    bus_trips_metadata_dictionary[column_headers]=column_desc_array[1]

    
tp_datetime_columns=[column_headers for column_headers, column_desc_array in bus_trips_data_dictionary.items() if column_desc_array[1] == 'datetime']
tp_categorical_columns=[column_headers for column_headers, column_desc_array in bus_trips_data_dictionary.items() if column_desc_array[1] == 'category']
tp_num_columns=[column_headers for column_headers, column_desc_array in bus_trips_data_dictionary.items() if column_desc_array[1] in ('numeric','int64','float64')]




    

#Dates for File Additions if needed
today_date=dt.datetime.now()

#DateTime objects
today_year=today_date.year
today_month=today_date.month
today_day=today_date.day

#Convert to ISO Standard for Filename
str_year=str(today_date.year)

#Month should have two digits
str_month=str(today_date.month)
if len(str_month)==1:
    str_month="0{}".format(str_month)

#Day should have two digits
str_day=str(today_date.day)
if len(str_day)==1:
    str_day="0{}".format(str_day)


str_today_date="{}-{}-{}".format(str_year,str_month,str_day)

datetime_format='%d-%b-%Y %H:%M:%S'

## 02. Exploration

In [1]:
def ingest_data(fp,delim,data_dictionary,chunks=10000000,pandas=False):
    """A function to read in CSV Data and Validate.
    
    Memory error after 50M rows"""

    print("Inside ingest_data({},dictionary)".format(fp))
    
    def print_shape(raw_df):
        """A function to print the shape of a dataframe"""
        #row_column data
        shape_of_df=raw_df.shape
        row_count=shape_of_df[0]
        column_count=shape_of_df[1]

        #print info to user
        row_column_print_statement='Your file contains: \n{} rows x {} columns.\n\n'
        row_column_print_statement=row_column_print_statement.format(row_count,column_count)
        print(row_column_print_statement)
        header_statement='The following columns are present:\n'

        #print the headers
        for header in raw_df.columns:
            header_statement+='"{}"\n'.format(header)

        print(header_statement)
        return
        
    def verify_schema(raw_df,data_dictionary):
        """A function to validate the schema of a dataframe"""
        
        match=False
        #check if the schema is correct
        if set(raw_df.columns)==set(data_dictionary.keys()) and len(raw_df.columns)==len(data_dictionary.keys()):
            print('The columns in this data sample match the schema')
            match=True

        else:
            print('The columns in this data sample do not match the schema')
            
        return match

        

    if pandas==True:
        #Valid Filepath
        if os.path.isfile(fp):

            raw_df = pd.DataFrame()
            chunk_count=0
            
            for chunk in pd.read_csv(fp,sep=delim,dtype=str,chunksize=chunks):
                chunk_count+=1
                print('On Chunk: {}'.format(chunk_count))
                raw_df = pd.concat([raw_df,chunk])
                
            display(raw_df)

            print_shape(raw_df)

            verify_schema(raw_df,data_dictionary)

            print("\n\n\nSample Data:\n\n\n")
            display(raw_df.head())
            
            return raw_df

        #Not Valid Filepath
        else:
            print("Invalid filepath - Correct the filepath and re-ingest")

            return
        
    elif pandas==False:
        
        #Valid Filepath
        if os.path.isfile(fp):

            #read_csv - Do Not Let Pandas Manipulate the Data First - Auto-assign is more memory intensive.
            raw_df=dask_df.read_csv(fp, sep=delim)
            display(raw_df)

            print_shape(raw_df)


            verify_schema(raw_df,data_dictionary)
            
            print("\n\n\nSample Data:\n\n\n")
            display(raw_df.head())

            return raw_df

        #Not Valid Filepath
        else:
            print("Invalid filepath - Correct the filepath and re-ingest")

            return
        
    else:
        print('No opinion on using Dask or Pandas - Defaulting to Dask')
        
        #Valid Filepath
        if os.path.isfile(fp):

            #read_csv - Do Not Let Pandas Manipulate the Data First - Auto-assign is more memory intensive.
            raw_df=dask_df.read_csv(fp, sep=delim)
            display(raw_df)

            print_shape(raw_df)


            verify_schema(raw_df,data_dictionary)

            print("\n\n\nSample Data:\n\n\n")
            display(raw_df.head())
            
            return raw_df
        
        #Not Valid Filepath
        else:
            print("Invalid filepath - Correct the filepath and re-ingest")

            return

In [5]:
def data_convert(df,types,columnlist,dt_format):
    """A function to convert all columns in a list into the appropriate type"""
    
    print("Inside data_convert()")
    
    ###Check if empty
    if len(df.index) != 0:
        
        ##Check if datetime or other
        if types=='datetime':
       
            ###Check if 0
            if len(columnlist)>0:
                print('Converting to {}'.format(types))
                
                for column in columnlist:
                    df[column]=df[column].apply(pd.to_datetime,format=dt_format,errors='ignore')
                
            else:
                print('No need to convert to: {}'.format(types))
              
        ###Numeric type
        elif types=='category':
            ###Check if 0
            if len(categorical_columns)>0:
                print('Converting to {}'.format(types))
                
                for column in columnlist:
                    df[column]=df[column].astype('category')
                
            ###Nothing to convert
            else:
                print('No need to convert')
                
        ###Numeric type
        elif types=='numeric':
            
            ###Check if 0
            if len(num_columns)>0:
                print('Converting to Numerical')
                
                for column in columnlist:
                    df[column]=df[column].apply(pd.to_numeric, errors='ignore')
                
            else:
                print('No need to convert')
                
        ###Other type - e.g. Boolean, string - Dont do anything - force the above types.
        else:
            print('Unknown type')
                
    ###Empty data set          
    else:
        print("Empty dataframe")

In [6]:

def missing_check(row):
    """Highlight rows with potential missing_values"""

    #Configuration Values
    col_to_check=10
    default_colour = 'white'
    flag_colour=''
    high_flag_colour_val='red'
    med_flag_colour_val='orange'
    low_flag_colour_val='yellow'
    val_to_check=0

    #Row length valid
    if len(row)>=col_to_check:

        #
        if row.values[col_to_check] == 'High':
            flag_colour = high_flag_colour_val
            
        elif row.values[col_to_check] == 'Medium':
            flag_colour = med_flag_colour_val
            
        elif row.values[col_to_check] == 'Low':
            flag_colour = low_flag_colour_val

        if flag_colour=='':
            colour=default_colour
        else:
            colour=flag_colour

        return ['background-color: {}'.format(colour)]*len(row.values)

    else:
        print('Row too short - Reconfigure Column Number')
        return ['background-color: {}'.format(default_colour)]*len(row.values)
    
def dt_missing_check(row):
    """Highlight rows with potential missing_values"""

    #Configuration Values
    col_to_check=9
    default_colour = 'white'
    flag_colour=''
    high_flag_colour_val='red'
    med_flag_colour_val='orange'
    low_flag_colour_val='yellow'
    val_to_check=0

    #Row length valid
    if len(row)>=col_to_check:

        #
        if row.values[col_to_check] == 'High':
            flag_colour = high_flag_colour_val
            
        elif row.values[col_to_check] == 'Medium':
            flag_colour = med_flag_colour_val
            
        elif row.values[col_to_check] == 'Low':
            flag_colour = low_flag_colour_val

        if flag_colour=='':
            colour=default_colour
        else:
            colour=flag_colour

        return ['background-color: {}'.format(colour)]*len(row.values)

    else:
        print('Row too short - Reconfigure Column Number')
        return ['background-color: {}'.format(default_colour)]*len(row.values)
    
def group_over_single_categories(df,categorical_columns,pdf_fn, save_fig=True):
    """A function to group over the categories"""
    
    print("Inside group_over_single_categories()")
    row_count=len(df)

    grouping_type={}
    timestamp_now=dt.datetime.timestamp(dt.datetime.now())
    
    #Dataframe is not empty, and there are categorical columns to group over:
    if df.empty==False and len(categorical_columns)>0:
        with PdfPages(pdf_fn) as pp: #lab
            column=''

            #Let's go through the category column type
            for column in categorical_columns:

                #Separator
                print('\n\n----------------------\n\n')
                agg_df=df.groupby([column]).agg({df.columns[0]:"count"})
                print(agg_df)
                agg_df=agg_df.reset_index()
                agg_df=agg_df.rename(columns={df.columns[0]:'Rows'})

                #Note: Could also do value_counts but I prefer that for graphing.
                agg_df['% Frequency']=100*(agg_df['Rows']/row_count)

                #Be explicit over what we're displaying
                print('Grouping over {} results in:\n'.format(column))

                #Display the result
                display(agg_df)
                #Graphing Section:

                figure = (
                            df[column]
                              .value_counts(dropna=True, normalize=True)
                              .plot(kind='bar'
                                    ,title='Count of values for {}'.format(column)
                                    , xlabel='Field Values'
                                    , ylabel='Count of Values'
                                    , figsize=(35,35)
                                   )
                 )
                #This grid style is from the sample Lab5 as I like how it looks
                plt.ylim([0,1])
                plt.grid(b=True, which='major', color='#666666', linestyle='-')
                plt.setp(figure.get_xticklabels(), ha="right", rotation=0)
                plt.minorticks_on()
                plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
                plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
                plt.show()
                grouping_type[column]=agg_df

                if save_fig:
                    pp.savefig(figure.get_figure())


                





    
    return grouping_type

def group_over_multi_categories(df,categorical_columns,pdf_fn,save_output=False,save_fig=False):
    """A function to group over all pairs of categories
    
    Warning: This can be memory intensive as we have (columnCount)C(2) pairings, so only run this if your device is able!"""
    
    print("Inside group_over_multi_categories()")
    row_count=len(df)
    grouping_type={}
    timestamp_now=dt.datetime.timestamp(dt.datetime.now())
    
    #Try run this
    try:
    
        #Dataframe is not empty, and there are categorical columns to group over:
        if df.empty==False and len(categorical_columns)>0:
            with PdfPages(pdf_fn) as pp:
                column=''
                second_column=''
                
                #Let's go through the category column type
                for column in categorical_columns:

                    #Second index, n^2
                    for second_column in categorical_columns:
                        multi_column=[column]


                        #Create a key to access - pipe delimited as columns contain _
                        grouping_key="{}|{}"

                        #No point in grouping the same column twice
                        if second_column!=column:
                            multi_column+=[second_column]
                            grouping_key=grouping_key.format(column,second_column)

                            #Separator
                            print('\n\n----------------------\n\n')
                            agg_df=df.groupby(multi_column).agg({df.columns[0]:"count"})
                            agg_df=agg_df.reset_index()
                            agg_df=agg_df.rename(columns={df.columns[0]:'Rows'})
                            agg_df['% Frequency']=100*(agg_df['Rows']/row_count)

                            #Be explicit over what we're displaying
                            print('Grouping over {} results in:\n'.format(grouping_key))

                            #Display the result
                            display(agg_df)
                            
                            #Graph
                            figure = (
                                        (df[multi_column]
                                              .dropna()
                                              .value_counts(normalize=True)
                                              .reset_index()
                                              .pivot_table(index=column,columns=second_column)
                                              .fillna(0))[0]
                                                      .plot(kind='bar'
                                                        , stacked=True
                                                        , title='Count of values for {} vs {}'.format(second_column,column)
                                                        , xlabel='Field Values'
                                                        , ylabel='Count of Values'
                                                        , figsize=(35,35)

                                                            
                                                           )
                                     )
                
                            #This grid style is from the sample Lab5 as I like how it looks
                            plt.ylim([0,1])
                            plt.grid(b=True, which='major', color='#666666', linestyle='-')
                            plt.setp(figure.get_xticklabels(), ha="right", rotation=0)
                            plt.minorticks_on()
                            plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
                            plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
                            plt.show()

                            

                            if save_fig:
                                pp.savefig(figure.get_figure())

                            #Only save if explicitly passed - This could kill your memory.
                            if save_output:
                                grouping_type[grouping_key]=agg_df

                        
    #Catch exceptions
    except Exception as exc:
        print("Function exception:\n")
        #check exception is memory error
        if exc==MemoryError:
            print("Sorry, your device is not able to run this function as you have hit a memory limit")
            
        print(exc)
        

    return grouping_type


def cat_missing_check_cleanse(row):
    """Highlight rows with potential missing_values"""

    #Configuration Values
    col_to_check=8
    default_colour = 'green'
    flag_colour=''
    high_flag_colour_val='red'
    med_flag_colour_val='orange'
    low_flag_colour_val='yellow'
    val_to_check=0

    #Row length valid
    if len(row)>=col_to_check:

        #
        if row.values[col_to_check] == 'High':
            flag_colour = high_flag_colour_val
            
        elif row.values[col_to_check] == 'Medium':
            flag_colour = med_flag_colour_val
            
        elif row.values[col_to_check] == 'Low':
            flag_colour = low_flag_colour_val

        if flag_colour=='':
            colour=default_colour
        else:
            colour=flag_colour

        return ['background-color: {}'.format(colour)]*len(row.values)

    else:
        print('Row too short - Reconfigure Column Number')
        return ['background-color: {}'.format(default_colour)]*len(row.values)
    
def stacked_group_over_multi_categories(df,categorical_columns,pdf_fn,save_output=False,save_fig=False):
    """A function to group over all pairs of categories
    
    Warning: This can be memory intensive as we have (columnCount)C(2) pairings, so only run this if your device is able!"""
    
    print("Inside group_over_multi_categories()")
    row_count=len(df)
    grouping_type={}
    timestamp_now=dt.datetime.timestamp(dt.datetime.now())
    
    #Try run this
    try:
    
        #Dataframe is not empty, and there are categorical columns to group over:
        if df.empty==False and len(categorical_columns)>0:
            with PdfPages(pdf_fn) as pp:
                column=''
                second_column=''
                
                #Let's go through the category column type
                for column in categorical_columns:

                    #Second index, n^2
                    for second_column in categorical_columns:
                        multi_column=[column]


                        #Create a key to access - pipe delimited as columns contain _
                        grouping_key="{}|{}"

                        #No point in grouping the same column twice
                        if second_column!=column:
                            multi_column+=[second_column]
                            grouping_key=grouping_key.format(column,second_column)

                            #Separator
                            print('\n\n----------------------\n\n')
                            agg_df=df.groupby(multi_column).agg({df.columns[0]:"count"})
                            agg_df=agg_df.reset_index()
                            agg_df=agg_df.rename(columns={df.columns[0]:'Rows'})
                            agg_df['% Frequency']=100*(agg_df['Rows']/row_count)

                            #Be explicit over what we're displaying
                            print('Grouping over {} results in:\n'.format(grouping_key))

                            #Display the result
                            display(agg_df)
                            agg_df=agg_df.reset_index()

                            sagg_df=(
                                    df
                                    .groupby([column])
                                    .agg({df.columns[0]:"count"})
                                    .reset_index()
                                    .rename(columns={df.columns[0]:'TotalRows'})
                                    )

                            join_df=agg_df.merge(sagg_df,left_on=column,right_on=column,suffixes=('_subbed','_group'))
                            join_df['% Stacked']=join_df['Rows']/join_df['TotalRows']

                            figure=((join_df
                                  .pivot_table(index=column,columns=second_column,values='% Stacked')
                                  .fillna(0))

                                          .plot(kind='bar'
                                            , stacked=True
                                            , title='Distribution of values for {} vs {}'.format(second_column,column)
                                            , xlabel='Field Values'
                                            , ylabel='Makeup of Values'
                                            , figsize=(35,35)

                            ))

                            #This grid style is from the sample Lab5 as I like how it looks
                            plt.ylim([0,1])
                            plt.grid(b=True, which='major', color='#666666', linestyle='-')
                            plt.setp(figure.get_xticklabels(), ha="right", rotation=0)
                            plt.minorticks_on()
                            plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
                            plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
                            plt.show()

                            

                            if save_fig:
                                pp.savefig(figure.get_figure())

                            #Only save if explicitly passed - This could kill your memory.
                            if save_output:
                                grouping_type[grouping_key]=agg_df

                        
    #Catch exceptions
    except Exception as exc:
        print("Function exception:\n")
        #check exception is memory error
        if exc==MemoryError:
            print("Sorry, your device is not able to run this function as you have hit a memory limit")
            
        print(exc)
        

    return grouping_type


def stacked_group_over_target_categories(df,categorical_columns,pdf_fn,save_output=False,save_fig=False):
    """A function to group over all pairs of categories and the target death_yn
    
    Warning: This can be memory intensive as we have so only run this if your device is able!"""
    
    print("Inside group_over_multi_categories()")
    row_count=len(df)
    grouping_type={}
    timestamp_now=dt.datetime.timestamp(dt.datetime.now())
    
    #Try run this
    try:
    
        #Dataframe is not empty, and there are categorical columns to group over:
        if df.empty==False and len(categorical_columns)>0:
            with PdfPages(pdf_fn) as pp:
                column=''
                second_column=''
                
                #Let's go through the category column type
                for column in categorical_columns:

                    #Second index, n^2
                    for second_column in ['death_yn']:
                        multi_column=[column]


                        #Create a key to access - pipe delimited as columns contain _
                        grouping_key="{}|{}"

                        #No point in grouping the same column twice
                        if second_column!=column:
                            multi_column+=[second_column]
                            grouping_key=grouping_key.format(column,second_column)

                            #Separator
                            print('\n\n----------------------\n\n')
                            agg_df=df.groupby(multi_column).agg({df.columns[0]:"count"})
                            agg_df=agg_df.reset_index()
                            agg_df=agg_df.rename(columns={df.columns[0]:'Rows'})
                            agg_df['% Frequency']=100*(agg_df['Rows']/row_count)

                            #Be explicit over what we're displaying
                            print('Grouping over {} results in:\n'.format(grouping_key))

                            #Display the result
                            display(agg_df)
                            agg_df=agg_df.reset_index()

                            sagg_df=(
                                    df
                                    .groupby([column])
                                    .agg({df.columns[0]:"count"})
                                    .reset_index()
                                    .rename(columns={df.columns[0]:'TotalRows'})
                                    )

                            join_df=agg_df.merge(sagg_df,left_on=column,right_on=column,suffixes=('_subbed','_group'))
                            join_df['% Stacked']=join_df['Rows']/join_df['TotalRows']

                            figure=((join_df
                                  .pivot_table(index=column,columns=second_column,values='% Stacked')
                                  .fillna(0))

                                          .plot(kind='bar'
                                            , stacked=True
                                            , title='Distribution of values for {} vs {}'.format(second_column,column)
                                            , xlabel='Field Values'
                                            , ylabel='Makeup of Values'
                                            , figsize=(35,35)

                            ))

                            #This grid style is from the sample Lab5 as I like how it looks
                            plt.ylim([0,1])
                            plt.grid(b=True, which='major', color='#666666', linestyle='-')
                            plt.setp(figure.get_xticklabels(), ha="right", rotation=0)
                            plt.minorticks_on()
                            plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
                            plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
                            plt.show()

                            

                            if save_fig:
                                pp.savefig(figure.get_figure())

                            #Only save if explicitly passed - This could kill your memory.
                            if save_output:
                                grouping_type[grouping_key]=agg_df

                        
    #Catch exceptions
    except Exception as exc:
        print("Function exception:\n")
        #check exception is memory error
        if exc==MemoryError:
            print("Sorry, your device is not able to run this function as you have hit a memory limit")
            
        print(exc)
        

    return grouping_type

In [25]:
def group_by_column(df,groupby_columns,agg_dict):
    """A function to group by columns given and aggregate according to a dictionary.
    
    Input: df, columns to group by, agg_dictionary
    """
    
    print("inside group_by_column(df,{},{})".format(groupby_columns,agg_dict))
    
    #Possible Errors
    error_dictionary={0:'No Error'
                     ,1:'The dataframe is empty'
                     ,2:"The columns to group by is empty or not a list"
                     ,3: 'The dictionary is empty'
                     ,4: 'The dataframe does not contain the required columns'
                      ,999: 'Uncaught exception'
                     }
    
    #Set as empty
    summary_df=pd.DataFrame()
    required_columns=[]
    
    error_code=0
    
    try:

        #Dictionary is non-empty
        if len(agg_dict)>0 and type(agg_dict)==dict:

            #df not empty
            if len(df)>0:

                #List and non-empty
                if type(groupby_columns)==list and len(groupby_columns)>0:
                    required_columns=list(df.columns)+list(agg_dict.keys())

                    #Required columns found
                    if set(required_columns).issubset(set(df.columns)):

                        #begin groupby - note: not catching summary issues as they are plentiful
                        summary_df=(df
                                        .groupby(groupby_columns)
                                        .agg(agg_dict)
                                        .reset_index()
                                    )


                    #Required columns not found    
                    else:
                        error_code=4
                        error_message=error_dictionary[error_code]
                        print(error_message)

                #Not a list or empty
                else:
                    error_code=2
                    error_message=error_dictionary[error_code]
                    print(error_message)

            #df is not empty
            else:
                error_code=1
                error_message=error_dictionary[error_code]
                print(error_message)

        #empty Dictionary
        else:
            error_code=3
            error_message=error_dictionary[error_code]
            print(error_message)
            
    except Exception as e:
        error_code=999
        print("Uncaught exception: {}".format(e))
        
    return [error_code,summary_df]

### 02.02 Begin Looking at the Trips dataset

Using Pandas to ingest the dataset and display some summary statistics

The features of the trips dataset are:

From a brief look it looks as if the the columns Ternderlot, Suppressed, justification might be all empty. Empty columns can be removed from the dataset.

The datasource feature could be a constant column so will not include any valuable information for our task, so i suggest it be removed.

There are some missing values in the feature actual departure time, which will raise some issues in the future, this feature might be important depending on what we choose to do, so we may want to find a way to repair the feature.

The feature "basin" could be a constant feature as well, using a similar argument to datasource , we can remove the column.

This would mean that form the "trips" dataset,

[DayOfService

TripID

LineID

ROuteID

Direction

PLannedTime_arr

PlannedTime_DEP

ActualTime_arr

actualtime_dep

last_update

note]

 would be the only columns remaining.

### 02.02 Looking at the Leavetime dataset

I will use dask to look at the dataset as it can load more things and is supposed to be faster for larger datasets.

# opening everything with dask

because using pandas crashes my connection with the server

## Reading and cehcking the data for irregularities

In [2]:
import dask.dataframe as dd

In [3]:
bus_leavetimes_df = dd.read_csv("./data/rt_leavetimes_DB_2018.txt", sep=";")

In [3]:
bus_leavetimes_df

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
npartitions=337,,,,,,,,,,,,,,,,,,
,object,object,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
bus_trip_df = dd.read_csv("./data/rt_trips_DB_2018.txt", sep=";")

In [5]:
bus_trip_df

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
npartitions=7,,,,,,,,,,,,,,,,
,object,object,int64,object,object,int64,int64,int64,float64,float64,object,float64,float64,float64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
LIST_CATEGORIES = []

bus_trip_df["DATASOURCE"] = bus_trip_df["DATASOURCE"].astype("category")
bus_trip_df["DIRECTION"] = bus_trip_df["DIRECTION"].astype("category")
bus_trip_df["BASIN"] = bus_trip_df["BASIN"].astype("category")
bus_trip_df["LINEID"] = bus_trip_df["LINEID"].astype("category")
bus_trip_df["ROUTEID"] = bus_trip_df["ROUTEID"].astype("category")

#float data
bus_trip_df["TRIPID"] = bus_trip_df["TRIPID"].astype("int64")
bus_trip_df["PLANNEDTIME_ARR"] = bus_trip_df["PLANNEDTIME_ARR"].astype("int64")
bus_trip_df["PLANNEDTIME_DEP"] = bus_trip_df["PLANNEDTIME_DEP"].astype("int64")
#bus_trip_df["NOTE"] = bus_trip_df["NOTE"].astype("int64")

#bus_trip_df["ACTUALTIME_ARR"] = bus_trip_df["ACTUALTIME_ARR"].astype("int64")
#bus_trip_df["ACTUALTIME_DEP"] = bus_trip_df["ACTUALTIME_DEP"].astype("int64")



From looking at the historical data after looking at the realtime, current data, we see that the tripid is formatted differently, 

In [7]:
# check the data is imported correctly
bus_trip_df.head(10)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"
5,DB,07-FEB-18 00:00:00,6248240,77A,77A_28,2,41648,37200,42019.0,37538.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2966500,"
6,DB,07-FEB-18 00:00:00,6251760,39,39_21,2,34768,28920,35709.0,28929.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2422850,"
7,DB,07-FEB-18 00:00:00,6262909,16,16_20,1,43936,38880,43721.0,38955.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3045261,"
8,DB,07-FEB-18 00:00:00,6251147,40D,40D_102,1,33558,30600,34540.0,31185.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2837123,"
9,DB,07-FEB-18 00:00:00,6249435,27B,27B_34,2,52682,49800,53281.0,49974.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2329338,"


In [8]:
# che k number of unique lines in the trips dataset
bus_trip_df["LINEID"].nunique().compute()

130

In [9]:
len(bus_leavetimes_df)

116949113

In [10]:
# the number of routes,
# contaisn different versions of the same line as the 
bus_trip_df["ROUTEID"].nunique().compute()

588

In [5]:
# collecting the trips dataset with respect to the Line and collecting all of the tripids in a list
filter_result = bus_trip_df.groupby('LINEID')['TRIPID'].apply(list).reset_index()

<ipython-input-5-183d341f903f>:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  filter_result = bus_trip_df.groupby('LINEID')['TRIPID'].apply(list).reset_index()


In [25]:
filter_result["LINEID"]

,LINEID,TRIPID
0,111,"[7321397, 6736637, 6736639, 6736647, 6736630, ..."
1,145,"[6636273, 6631234, 6641668, 6643671, 6643699, ..."
2,15D,"[7175345, 8070285, 8052192, 7168575, 8050929, ..."
3,238,"[7146929, 8098733, 7328510, 7328508, 7319958, ..."
4,25X,"[8068486, 8104083, 8052715, 8124060, 6737359, ..."


In [12]:
# list the routes associated with the line
filter_route_line_result = bus_trip_df.groupby('LINEID')['ROUTEID'].apply(list).reset_index()

<ipython-input-12-81d12947e224>:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  filter_route_line_result = bus_trip_df.groupby('LINEID')['ROUTEID'].apply(list).reset_index()


In [13]:
filter_route_line_result.compute()

,LINEID,ROUTEID
0,111,"[111_8, 111_8, 111_9, 111_10, 111_7, 111_8, 11..."
1,145,"[145_105, 145_105, 145_105, 145_102, 145_105, ..."
2,15D,"[15D_63, 15D_62, 15D_65, 15D_62, 15D_62, 15D_6..."
3,238,"[238_11, 238_15, 238_15, 238_15, 238_11, 238_1..."
4,25X,"[25X_10, 25X_11, 25X_11, 25X_11, 25X_12, 25X_1..."
...,...,...
17,66,"[66_11, 66_13, 66_13, 66_11, 66_11, 66_13, 66_..."
18,75,"[75_18, 75_19, 75_17, 75_17, 75_17, 75_19, 75_..."
19,79A,"[79A_27, 79A_27, 79A_27, 79A_27, 79A_27, 79A_2..."
20,7A,"[7A_85, 7A_85, 7A_88, 7A_86, 7A_87, 7A_87, 7A_..."


In [14]:
len(filter_route_line_result)

130

In [15]:
list_routes = list(filter_route_line_result["ROUTEID"].compute())

In [16]:
from collections import Counter

Of the 102 lines it has this number of routes, and the frequencies

In [17]:
print(Counter(list_routes[0]))

Counter({'111_7': 4005, '111_8': 2886, '111_9': 719, '111_10': 238})


In [18]:
%time len(filter_result)

CPU times: user 14.6 s, sys: 1.69 s, total: 16.3 s
Wall time: 10.5 s


130

Tells us that there are 130 unique lines, these include their own routes , the routes differ by the different starting points or different ending points

In [7]:
%time filter_result.head()

CPU times: user 8.62 s, sys: 2.08 s, total: 10.7 s
Wall time: 8.73 s


,LINEID,TRIPID
0,102,"[6121819, 6858629, 6855177, 6855178, 6646206, ..."
1,104,"[6856598, 6647566, 6647571, 6276763, 6759749, ..."
2,116,"[7159266, 6291853, 6099533, 6088623, 6291512, ..."
3,118,"[6752981, 6118699, 6665591, 7147145, 6242604, ..."
4,120,"[6115001, 6115928, 6852405, 6860169, 6853319, ..."


In [31]:
# get the list of bus routes
bus_routes = list(filter_result.LINEID.compute())

In [29]:
filter_result.LINEID.compute()

0     111
1     145
2     15D
3     238
4     25X
     ... 
17     66
18     75
19    79A
20     7A
21     84
Name: LINEID, Length: 130, dtype: object

In [32]:
# we have the unique bus routes
print(bus_routes)

['111', '145', '15D', '238', '25X', '27A', '27X', '46E', '49', '51D', '65', '65B', '67', '7', '76A', '77X', '114', '14', '15A', '16C', '185', '25', '27', '270', '33', '39', '39A', '40', '40E', '41', '41B', '45A', '68X', '76', '77A', '83', '104', '116', '120', '220', '236', '31', '31D', '33B', '4', '40B', '42D', '46A', '53', '54A', '66X', '68', '70D', '118', '122', '123', '18', '184', '25B', '31B', '40D', '41A', '42', '43', '56A', '61', '67X', '84X', '1', '13', '150', '151', '161', '25A', '29A', '31A', '33A', '33X', '37', '47', '66A', '66B', '69X', '7D', '9', '102', '130', '142', '14C', '15', '16', '17A', '26', '32', '33D', '33E', '38B', '38D', '44', '44B', '51X', '68A', '69', '70', '79', '7B', '83A', '84A', '11', '140', '15B', '16D', '17', '239', '25D', '27B', '32X', '38', '38A', '39X', '41C', '41D', '41X', '59', '63', '66', '75', '79A', '7A', '84']


In [6]:
#list the trips that are associated iwth route 102, route chosen randomly
# testing for the first line in the list
%time line_102 = list(bus_trip_df["TRIPID"][bus_trip_df["LINEID"] == "102"].compute())

CPU times: user 6.37 s, sys: 959 ms, total: 7.33 s
Wall time: 5.29 s


In [23]:
line_102

[7764193,
 7761736,
 7762172,
 6277493,
 6277492,
 6266653,
 7761744,
 7100366,
 7100367,
 7101507,
 6278971,
 6266657,
 6266656,
 6266658,
 6278972,
 6620297,
 6617210,
 6623766,
 6620258,
 6389409,
 6387381,
 6387383,
 7099681,
 7101512,
 6613665,
 6617208,
 6617209,
 6620257,
 6620294,
 6387380,
 8024207,
 8024212,
 6389406,
 7101518,
 7110511,
 7110509,
 7110513,
 7025238,
 7025241,
 6266654,
 6266652,
 6277494,
 6278969,
 8017278,
 8016530,
 8017981,
 8027705,
 8017277,
 8017980,
 8017651,
 6401158,
 6401168,
 8093343,
 6266649,
 8085867,
 8082702,
 8090455,
 8090453,
 8093348,
 8091211,
 8086416,
 7025103,
 7025120,
 7025109,
 6266648,
 8016526,
 8017979,
 7161974,
 6620306,
 6625125,
 6620304,
 8093353,
 8091214,
 8086419,
 8086410,
 8091204,
 8082697,
 7025094,
 7025147,
 7025105,
 7025123,
 7025111,
 7025154,
 8024211,
 8017653,
 8016529,
 8027703,
 8027712,
 6389450,
 6389451,
 6389411,
 6387385,
 7769069,
 7772164,
 7772170,
 7761743,
 8090447,
 8091202,
 8093342,
 7025226,


In [14]:
#dataframe is the datframe from which the selection is chosen from
# route is the string representaztion of the route
def route_selection(dataframe, route):
    route_leavetimes = dataframe.map_partitions(lambda x: x[x.TRIPID.isin(route)])
    return route_leavetimes

In [8]:
%time line_102 = list(bus_trip_df["TRIPID"][bus_trip_df["LINEID"] == "102"].compute())

CPU times: user 7.07 s, sys: 1.15 s, total: 8.22 s
Wall time: 4.58 s


In [26]:
print(route_102_8)

[6277493, 6277492, 6266653, 7100366, 7101507, 6266656, 6266658, 6278972, 6620297, 6617210, 6623766, 6620258, 6389409, 6387381, 6387383, 6613665, 6617208, 8024212, 8017278, 8016530, 8017981, 6401158, 6401168, 6266649, 8085867, 8082702, 8090455, 8090453, 8093348, 8091211, 8086416, 7025103, 7025120, 7025109, 8016526, 8017979, 7025094, 7025147, 7025105, 7025123, 7025111, 7025154, 8027712, 6389450, 6389451, 6389411, 6387385, 7769069, 7772164, 7772170, 7761743, 6620305, 6625124, 6625126, 6620299, 6389416, 6389414, 6387389, 7148431, 6390169, 6403108, 6613668, 6625122, 6625116, 6390167, 6389407, 7135164, 6266651, 6278964, 6278970, 6277495, 7025155, 7025112, 7025095, 7148438, 7145597, 7148440, 6389453, 6387387, 6230505, 8017652, 8017276, 8016528, 8027704, 6623773, 6236035, 6229364, 8017984, 8016534, 8027710, 8017282, 8017986, 8024214, 8027712, 8017657, 6620303, 7773579, 7145589, 7148433, 8082698, 6230673, 8017284, 8024216, 8017988, 8017659, 8027714, 8024218, 7145593, 7135161, 7148435, 7135162, 

In [15]:
# testing if above function works
%time route_102_df = route_selection(bus_leavetimes_df, line_102)


CPU times: user 24.2 ms, sys: 40 µs, total: 24.3 ms
Wall time: 22.6 ms


In [17]:
%time route_102_df.head()

CPU times: user 747 ms, sys: 99.3 ms, total: 846 ms
Wall time: 948 ms


,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE
3755,01-JAN-18 00:00:00,5957731,2,935,77743,77743,77715,77715,2868314,NaN,NaN,08-JAN-18 17:21:10
3756,01-JAN-18 00:00:00,5968222,2,935,82843,82843,82844,82844,2534829,NaN,NaN,08-JAN-18 17:21:10
3757,01-JAN-18 00:00:00,5957733,2,935,84643,84643,84643,84643,2868314,NaN,NaN,08-JAN-18 17:21:10
3758,01-JAN-18 00:00:00,5968214,1,4381,42300,42300,42326,42326,1001151,NaN,NaN,08-JAN-18 17:21:10
3759,01-JAN-18 00:00:00,5957726,1,4381,48300,48300,48298,48298,1000198,NaN,NaN,08-JAN-18 17:21:10


In [18]:
%time len(route_102_df)

CPU times: user 5min 51s, sys: 47.7 s, total: 6min 39s
Wall time: 2min 38s


998739

In [ ]:
stations_list_102_8 = list(route_102_8_df.STOPPOINTID.compute())


In [ ]:
list_route_102_8_stops = list(Counter(stations_list_102_8).keys())
for stop in list_route_102_8_stops:
    print(stop)

In [ ]:
#checking if the lines were pulled correctly
print(line_102)

In [10]:
# change datatypes of the leavetimes dataset
bus_leavetimes_df["DATASOURCE"] = bus_leavetimes_df["DATASOURCE"].astype("category")
bus_leavetimes_df["DAYOFSERVICE"] = dd.to_datetime(bus_leavetimes_df["DAYOFSERVICE"])
bus_leavetimes_df["LASTUPDATE"] = dd.to_datetime(bus_leavetimes_df["LASTUPDATE"])


In [11]:
#the datatypes of the features should be correct
bus_leavetimes_df.dtypes

DATASOURCE               category
DAYOFSERVICE       datetime64[ns]
TRIPID                      int64
PROGRNUMBER                 int64
STOPPOINTID                 int64
PLANNEDTIME_ARR             int64
PLANNEDTIME_DEP             int64
ACTUALTIME_ARR              int64
ACTUALTIME_DEP              int64
VEHICLEID                   int64
PASSENGERS                float64
PASSENGERSIN              float64
PASSENGERSOUT             float64
DISTANCE                  float64
SUPPRESSED                float64
JUSTIFICATIONID           float64
LASTUPDATE         datetime64[ns]
NOTE                      float64
dtype: object

# looking at the data and cleaning it accordingly

performing the min and max of the features can give us insight into the features themselves, so since the dataypes should all be correct it will point us towards outliers or irregular data

In [36]:
%time bus_leavetimes_df.max().compute()

CPU times: user 7min 44s, sys: 1min 18s, total: 9min 2s
Wall time: 2min 55s


DAYOFSERVICE       2018-12-31 00:00:00
TRIPID                         8592207
PROGRNUMBER                        109
STOPPOINTID                       7692
PLANNEDTIME_ARR                  91680
PLANNEDTIME_DEP                  91680
ACTUALTIME_ARR                   97177
ACTUALTIME_DEP                   97177
VEHICLEID                      3394131
PASSENGERS                         NaN
PASSENGERSIN                       NaN
PASSENGERSOUT                      NaN
DISTANCE                           NaN
SUPPRESSED                           1
JUSTIFICATIONID            4.84981e+17
LASTUPDATE         2019-01-16 18:27:21
NOTE                               NaN
dtype: object

In [37]:
%time bus_leavetimes_df.min().compute()

CPU times: user 7min 32s, sys: 1min 30s, total: 9min 2s
Wall time: 2min 58s


DAYOFSERVICE       2018-01-01 00:00:00
TRIPID                         5955221
PROGRNUMBER                          1
STOPPOINTID                          2
PLANNEDTIME_ARR                  16200
PLANNEDTIME_DEP                  16200
ACTUALTIME_ARR                   15974
ACTUALTIME_DEP                   15974
VEHICLEID                      1000110
PASSENGERS                         NaN
PASSENGERSIN                       NaN
PASSENGERSOUT                      NaN
DISTANCE                           NaN
SUPPRESSED                           0
JUSTIFICATIONID                 177856
LASTUPDATE         2018-01-08 17:21:10
NOTE                               NaN
dtype: object

In [11]:
# these items are all empty or constant
bus_leavetimes_df = bus_leavetimes_df.drop(["DATASOURCE", "PASSENGERS", "PASSENGERSIN", "PASSENGERSOUT", "DISTANCE", "NOTE"], axis=1)

ValueError: Metadata inference failed in `drop_by_shallow_copy`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KeyError("['DATASOURCE' 'PASSENGERS' 'PASSENGERSIN' 'PASSENGERSOUT' 'DISTANCE'\n 'NOTE'] not found in axis")

Traceback:
---------
  File "/home/team10/miniconda3/lib/python3.9/site-packages/dask/dataframe/utils.py", line 176, in raise_on_meta_error
    yield
  File "/home/team10/miniconda3/lib/python3.9/site-packages/dask/dataframe/core.py", line 5612, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/home/team10/miniconda3/lib/python3.9/site-packages/dask/dataframe/utils.py", line 710, in drop_by_shallow_copy
    df2.drop(columns=columns, inplace=True, errors=errors)
  File "/home/team10/m

In [12]:
bus_leavetimes_df.dtypes

DAYOFSERVICE        object
TRIPID               int64
PROGRNUMBER          int64
STOPPOINTID          int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR       int64
ACTUALTIME_DEP       int64
VEHICLEID            int64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
dtype: object

From the calculating the max values we can see that the max value of the features: [ passeengers* , distance , note] are all none, which means that the columns are empty which means that we can remove them.

The feature "datasource" should be a constant columns as all the data comes form a database.

We can split the leavetimes dataset with respect to the route of the bus, so that only the information of each route is displayed. This will make the dataset smaller and easier to maniupulate and work with.

In [17]:
import sys
!{sys.executable} -m pip install tqdm

In [33]:
from tqdm import tqdm

In [ ]:
print(tqdm)

In [ ]:
# splitting the data with respect to the lineid, the route id are different versions of the same line
filename = "~/data/leavetimes_split_by_route/route{}.csv"
for i in tqdm(range(130)):
    route_leavetimes = bus_leavetimes_df.map_partitions(lambda x: x[x.TRIPID.isin([filter_result["LINEID"] == bus_routes[i]])])
    route_leavetimes.to_csv(filename.format(bus_routes[i]))

  0%|          | 0/130 [00:00<?, ?it/s]

we loop through each route and get the 

In [34]:
filename = "~/data/leavetimes_split_by_route/route{}.csv"

i = 0
for bus in tqdm(bus_routes):
    print(i)
    i += 1
    line_tripid = list(bus_trip_df["TRIPID"][bus_trip_df["LINEID"] == bus].compute())
    single_route_df = route_selection(bus_leavetimes_df, line_tripid)
    single_route_df.to_csv(filename.format(bus))

  0%|          | 0/130 [00:00<?, ?it/s]

0


  1%|          | 1/130 [02:44<5:54:36, 164.93s/it]

1


  2%|▏         | 2/130 [06:35<7:13:38, 203.27s/it]

...

 98%|█████████▊| 128/130 [6:43:17<06:11, 185.88s/it]

128


 99%|█████████▉| 129/130 [6:46:14<03:03, 183.31s/it]

129


100%|██████████| 130/130 [6:49:05<00:00, 188.81s/it]


In [24]:
%time filter_result["TRIPID"][filter_result["LINEID" =="102"]]

KeyError: False